# AWS - RDS MySQL
### Paquetes requeridos
Librería de Python necesaria para establecer conexión a una base de datos MySQL

### Credenciales
Para acceder a la BD necesitaremos tres datos:
1. Endpoint/host: la dirección donde se encuentra el servidor
2. Usuario
3. Contraseña

In [12]:
from getpass import getpass
from mysql.connector import connect, Error

def connect_db(host, user):
  try:
    # Si la conexion se da satisfactoriamente, tendremos 
    # un mensaje de éxito. 
     conn = connect(
                    host=host,
                    user=user,
                    password=getpass("Enter password: ")
                     )

         
     print('SUCCESSFUL\n\n')
     return conn
  except Error as e:
    # En caso de tener un error de conexión, lo mostramos
    print(e)

### Conexion BD

In [16]:
host = "database-1.cdtvtmfza8iy.eu-west-3.rds.amazonaws.com"
user = "admin"

In [17]:
conn = connect_db(host, user)

SUCCESSFUL




### Version DB

In [18]:
cursor = conn.cursor()
cursor.execute("SELECT VERSION()")
cursor.fetchall()

[('8.0.28',)]

### Creación de DB

In [20]:
def show_db(cursor):
    cursor.execute("SHOW DATABASEs")
    return cursor.fetchall()

show_db(cursor)

[('information_schema',), ('mysql',), ('performance_schema',), ('sys',)]

### Creación de tablas

In [34]:
cursor.execute("DROP DATABASE IF EXISTS country_data")
cursor.execute("CREATE DATABASE country_data")

In [35]:
cursor.execute("USE country_data")

In [36]:
cursor.execute("CREATE TABLE country(name TEXT, continent TEXT, population INT, gdp INT)")

In [37]:
def show_tables(cursor):
    cursor.execute("SHOW TABLES")
    return cursor.fetchall()

show_tables(cursor)

[('country',)]

### Seleccionar la BD

### Insertar datos

In [38]:
cursor.execute("INSERT INTO country  (name, continent, population, gdp) VALUES ('spain', 'europe', 47, 2.7)")
conn.commit()

### Leer datos

In [39]:
import pandas as pd 

In [40]:
df = pd.read_csv('country_data.csv', sep=";")
df

,name,continent,population,gdp
0,France,Europe,70,2.7
1,Japan,Asia,126,4.9
2,Argentina,America,44,0.5
3,Brasil,America,209,1.8


In [42]:
from sqlalchemy import create_engine
pwd = getpass("Introduce contraseña:")
engine = create_engine(f"mysql+pymysql://{user}:{pwd}@{host}/country_data")

In [43]:
df.to_sql("country", con=engine, if_exists="append", index=False)

In [44]:
cursor.execute("SELECT * FROM country")
cursor.fetchall()

[('spain', 'europe', 47, 3),
 ('France', 'Europe', 70, 3),
 ('Japan', 'Asia', 126, 5),
 ('Argentina', 'America', 44, 0),
 ('Brasil', 'America', 209, 2)]

### Guardar los cambios
Hay que ejecutar el commit antes de cerrar la sesión de la BD para que se guarden todos los cambios

### Excepciones
Se recomienda rodear este tipo de sentencias con un try/except, ya que suele ser software productivo y tiene que saber manejar fallos

In [ ]:
try:
    sql = '''SELECT * FROM country'''
    cursor.execute(sql)
    cursor.fetchall()
except Exception as e:
    print(e)

### Insertar datos de un CSV

In [ ]:
# Creamos el CSV
import pandas as pd

df = pd.DataFrame({
    'name': ['France', 'Japan', 'Argentina', 'Brasil'],
    'continent': ['Europe', 'Asia', 'America', 'America'],
    'population': [70, 126, 44, 209],
    'gdp': [2.7, 4.9, 0.5, 1.8]
})

df.to_csv('country_data.csv', sep=';', index=False)

### Cerrar la conexión cuando acabemos

In [ ]:
conn.close()